In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import UnstructuredPDFLoader, PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SupabaseVectorStore
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from supabase import create_client
import logging
import os
import psycopg2
import tools_restaurant

# Configuración inicial
load_dotenv()
DB_CONNECTION = os.getenv("DB_CONNECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL")
TOOLS=tools_restaurant.TOOLS

In [2]:

chat = ChatOpenAI(model="gpt-4o", temperature=0)

In [3]:
embeddings = OpenAIEmbeddings()  # Inicializar embeddings

In [4]:
SYSTEM_TEMPLATE_2= """
Eres un asistente virtual del concesionario distoyota, que se encargara de brindar informacion sobre los carros que necesite el usuario.
El usuario podrá estar interesado en  : 
    1. Un vehiculo nuevo
    2. Un vehiculo usado
    3. Repuestos/accesorios
    4. Realizar test drive
    5. Agendar mantenimiento
Te vas a basar en la siguiente información :
<context>
{context}
</context>
"""

In [5]:
from langchain_community.vectorstores import FAISS
def create_from_directory(file_directory):
    embeddings=OpenAIEmbeddings()
    data=[]
    for file in os.listdir(file_directory):
        path=os.path.join(file_directory,file)
        loader=PDFMinerLoader(path)
        data+=loader.load()

        logging.info(f"Documento cargado desde el archivo {path}")

    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

    all_splits=text_splitter.split_documents(data)

    logging.info("Documentos spliteados")
    return all_splits

#Take a directory and create/add vectors to the vector store
def add_pdfs_from_directory(file_directory):
    embeddings=OpenAIEmbeddings()
    try:
        vector_store=FAISS.load_local("db",embeddings)
        logging.info("Vector store cargada")
        all_splits=create_from_directory(file_directory)
        vector_store.add_documents(all_splits,embeddings)
        logging.info("Documentos añadidos")
    except :
        all_splits=create_from_directory(file_directory)
        logging.info("Documentos añadidos y creados")
        vector_store=FAISS.from_documents(all_splits,embeddings)
        logging.info("Vector store creada")
    vector_store.save_local("db")

    logging.info("Vector store guardada")
    return vector_store

In [6]:
def get_session_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

# Create a Prompt template with a LLM model
def create_chain_agent():
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_TEMPLATE_2),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(chat, TOOLS, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=TOOLS, verbose=True)
    return agent_executor


def get_chat(chain):
    """ Configura la cadena con historial de mensajes. """
    return RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [7]:
# Generates a response based on an input
def run_chain(message_body, wa_id, context, conversation_chain):
    message = conversation_chain.invoke({"input": message_body, "context": context}, {"configurable": {"session_id": wa_id}})
    logging.info(f"Generated message: {message['output']}")
    return message['output']

# Delete messages from the message history
def trim_messages(messages, conversation_limit=10):
    if len(messages) > conversation_limit:
        return messages[-conversation_limit:]
    return messages

In [8]:
agent_executor = create_chain_agent()

In [9]:
agent_executor_with_message_trimming = (RunnablePassthrough.assign(chat_history=lambda x: trim_messages(x["chat_history"]))
 | agent_executor)

# Get the session history
conversation_chain = get_chat(agent_executor_with_message_trimming)

In [10]:
db = add_pdfs_from_directory("../../data/")

In [11]:
loader=PDFMinerLoader("../../data/Ficha-tecnica-Yaris.pdf")
data=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

all_splits=text_splitter.split_documents(data)

In [42]:
from langchain_community.document_loaders import PDFPlumberLoader
loader=PDFPlumberLoader("../../data/Ficha-tecnica-Yaris.pdf")
data=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
all_splits=text_splitter.split_documents(data)
db=FAISS.from_documents(all_splits,embedding=OpenAIEmbeddings())

In [44]:
input="dame todas las caracteristicas del vehiculo"
context=db.similarity_search(input)
p=conversation_chain.invoke({"input":input,"context": context},{"configurable":{"session_id" : "1"}})



> Entering new AgentExecutor chain...
Claro, aquí tienes las características del Toyota Yaris Sport HB:

### Seguridad Activa
- **Asistentes de Conducción**: VSC (Control de Estabilidad del Vehículo) + TRC (Control de Tracción) + HAC (Asistente de Arranque en Pendientes)
- **Sensor de Luces**: Sí, con función "Sígueme a casa"
- **Cámara de Reversa**: Disponible en algunas versiones
- **Desempañador Trasero**: Sí
- **Sistema de Pre-Colisión**: Disponible en algunas versiones
- **Alerta de Salida de Carril**: Disponible en algunas versiones

### Seguridad Pasiva
- **Estructura Carrocería**: GOA (Global Outstanding Assessment)
- **Cinturones Frontales**: 2 cinturones de 3 puntos con ELR (Retractor de Bloqueo de Emergencia), pretensor, limitador de fuerza y ajuste de altura
- **Sistema ISOFIX**: Sí

### Exterior
- **Spoiler**: Sí, en la puerta trasera y del color de la carrocería
- **Manijas Exteriores de las Puertas**: Del color de la carrocería
- **Techo Corredizo**: Disponible en algu